In [ ]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
import astropy.io.fits as pf
import sys, glob

thedir='/Users/cslage/Research/LSST/code/auxtel/ats_data/20190808/'
%cd $thedir


In [ ]:
files = glob.glob('AT_O_20190808_000???.fits')
name = files[0][18:20]
print(files)
print(name)


In [ ]:
xstart = 505
xstop = 542
ov_start = 512
ystart = 200
ystop = 1800
xaxis = np.linspace(xstart,xstop-1,xstop-xstart)

hdulist = pf.open(files[0], mode='readonly', do_not_scale_image_data=True)
hdr18=hdulist[18].header
Vph = hdr18['PCK_HIP']
Vpl = hdr18['PCK_LOWP']
Vsh = hdr18['SCK_HIP']
Vsl = hdr18['SCK_LOWP']
Vog = hdr18['OGOV']
Vbb = hdr18['HVBIAS_V']

for file in [files[0]]:
    print(file)
    name = file[18:20]
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
    plt.figure(figsize=(15,15))
    plt.subplots_adjust(hspace = 0.5, wspace = 0.5)
    plt.suptitle("Serial CTE by segment - ITL AuxTel New CCD\n Vph=%.2f; Vpl=%.2f; Vsh=%.2f; Vsl=%.2f; Vog=%.2f; Vbb=%.2f"%(Vph,Vpl,Vsh,Vsl,Vog,Vbb), fontsize = 16)
    for segment in range(16):
            #try:
            plt.subplot(4,4,segment+1)
            imhdr=hdulist[segment+1].header
            extname = imhdr['EXTNAME']
            plt.title("CTE - %s"%extname)
            data = np.array(hdulist[segment+1].data + 32768, dtype = np.int32)
            flat_overscan = np.mean(np.array(data[:,xstop-8:xstop]),axis = 1)
            cte_data = ((np.transpose(np.transpose(data) - flat_overscan))[ystart:ystop,:].mean(axis=0))[xstart:xstop]
            cte_std = ((np.transpose(np.transpose(data) - flat_overscan))[ystart:ystop,:].std(axis=0) / np.sqrt(float(ystop-ystart)))[xstart:xstop]
            cti = np.median((np.transpose(np.transpose(data) - flat_overscan))[ystart:ystop,ov_start]\
            / (np.transpose(np.transpose(data) - flat_overscan))[ystart:ystop,ov_start-1]) / ov_start

            plt.scatter(xaxis, cte_data, color = 'blue')
            plt.errorbar(xaxis, cte_data, yerr=cte_std)
            plt.text(xstart + 8, 20000, "CTI = %.2E"%cti, fontsize = 12)
            plt.yscale('log',nonposy='clip')
            plt.ylim(.01, 200000)
            plt.xticks([510,520,530,540])
            plt.xlabel("X (First overscan = 512)")
            plt.ylabel("Signal(ADU)")
            print("Segment %s done"%extname)
            #except:
            #continue
    plt.savefig(thedir+"Serial_CTE_%s_08Aug19.pdf"%name)
    plt.clf()
